In [52]:
# task4
# Впишите ваш логин. Например "vpetrov"
user = "a.marchenko"
user

'a.marchenko'

In [53]:
# В данной ячейке параметры не меняем! 
from pyspark.sql.types import *
import pyspark.sql.functions as sf

current_dt = "2023-03-01"

demography_path = "/user/{}/data3/ok/coreDemography".format(user)
country_path = "/user/{}/data3/ok/geography/countries.csv".format(user)

city_path = "/home/a.marchenko/data/data3/rosstat/city.csv"
product_path = "/home/a.marchenko/data/data3/rosstat/product.csv"
price_path = "/home/a.marchenko/data/data3/rosstat/price"
products_for_stat_path = "/home/a.marchenko/data/data3/rosstat/products_for_stat.csv"
city_rs_path = "/home/a.marchenko/data/data3/ok/geography/rs_city.csv"

# Путь до результата
output_path1 = "/user/{}/task4/price_stat".format(user)
output_path2 = "/user/{}/task4/ok_dem".format(user)
output_path3 = "/user/{}/task4/product_stat".format(user)

In [ ]:
import os
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

os.environ["HADOOP_CONF_DIR"]="/etc/hadoop/conf"
os.environ["HIVE_HOME"]="/usr/lib/hive"
os.environ["METASTORE_PORT"]="9083"
# os.environ["SPARK_HOME"]="/usr/hdp/current/spark2-client"
# os.environ["JAVA_HOME"]="/usr/java/jdk1.8.0_191/jre"

spark = pyspark.sql.SparkSession.builder\
    .master("yarn")\
    .appName("a_marchenko_hw")\
    .config("spark.executor.instances", "1")\
    .config("spark.executor.memory", "1G")\
    .config("spark.executor.cores", "2")\
    .config("spark.dynamicAllocation.enabled", "false")\
    .config("spark.dynamicAllocation.executorIdleTimeout", "300s")\
    .config("spark.dynamicAllocation.maxExecutors", "1000")\
    .config("spark.driver.memory", "1G")\
    .config("spark.driver.maxResultSize", "1G")\
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")\
    .config("spark.kryoserializer.buffer.max", "1024m")\
    .enableHiveSupport()\
    .getOrCreate()

spark_context = spark.sparkContext

24/04/27 15:03:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/27 15:03:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/04/27 15:03:38 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/04/27 15:03:38 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [ ]:
print("""
Urls:
Yarn       http://91.219.226.252:8088/cluster/scheduler
Spark      http://91.219.226.252:8088/proxy/{app_id}/stages/
App info   http://91.219.226.252:8088/cluster/app/{app_id}/
""".format(app_id=spark_context.applicationId))

### Задание 1

In [5]:
# Читаем данные
CityDF = (
    spark.read
        .option("header", "false")
        .option("sep", ";")
        .csv(city_path)
)

ProductDF = (
    spark.read
        .option("header", "false")
        .option("sep", ";")
        .csv(product_path)
)

PriceDF = (
    spark.read
        .option("header", "false")
        .option("sep", ";")
        .csv(price_path)
)

ProductsForStatDF = (
    spark.read
        .option("header", "false")
        .option("sep", ";")
        .csv(products_for_stat_path)
)

In [6]:
ProductsForStatDF.printSchema()

root
 |-- _c0: string (nullable = true)



In [7]:
ProductsForStatDF.show()

+---+
|_c0|
+---+
|355|
|446|
|114|
|115|
|117|
|108|
|341|
|120|
|113|
|449|
|454|
|453|
|448|
|396|
|397|
|456|
|455|
|458|
|457|
|326|
+---+
only showing top 20 rows



In [8]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import regexp_replace

# Приводим типы и задаем названия полей.
ProductDF = (
    ProductDF
    .select(
        sf.col("_c1").cast(IntegerType()).alias("product_id"),
        sf.col("_c0").alias("product")
    )
)

CityDF = (
    CityDF
    .select(
        sf.col("_c1").cast(IntegerType()).alias("city_id"),
        sf.col("_c0").alias("city")
    )
)

PriceDF = PriceDF.withColumn('_c2', regexp_replace('_c2', ',', '.'))

PriceDF = (
    PriceDF
    .select(
        sf.col("_c0").cast(IntegerType()).alias("city_id"),
        sf.col("_c1").cast(IntegerType()).alias("product_id"),
        sf.col("_c2").cast(FloatType()).alias("price")
    )
)

ProductsForStatDF = (
    ProductsForStatDF
    .select(
        sf.col("_c0").cast(IntegerType()).alias("product_id")
    )
)



In [9]:
CityDF.printSchema()
CityDF.show()

ProductDF.printSchema()
ProductDF.show()

PriceDF.printSchema()
PriceDF.show()

ProductsForStatDF.printSchema()
ProductsForStatDF.show()

root
 |-- city_id: integer (nullable = true)
 |-- city: string (nullable = true)

+-------+----------------+
|city_id|            city|
+-------+----------------+
|      1|        Белгород|
|      2|          Губкин|
|      3|    Старый Оскол|
|      4|          Брянск|
|      5|           Навля|
|      6|          Клинцы|
|      7|      Новозыбков|
|      8|        Владимир|
|      9|Гусь-Хрустальный|
|     10|          Ковров|
|     11|           Муром|
|     12|         Воронеж|
|     13|    Борисоглебск|
|     14|         Россошь|
|     15|         Иваново|
|     16|         Кинешма|
|     17|          Калуга|
|     18|        Людиново|
|     19|         Обнинск|
|     20|        Кострома|
+-------+----------------+
only showing top 20 rows

root
 |-- product_id: integer (nullable = true)
 |-- product: string (nullable = true)

+----------+--------------------+
|product_id|             product|
+----------+--------------------+
|         1|     Фарш мясной, кг|
|         2|Пельмени

In [10]:
PriceDF.count()

145976

In [11]:
# Выбираем продукты для статистики 
PriceJoinedDF = (
    PriceDF
    .join(ProductsForStatDF, PriceDF.product_id == ProductsForStatDF.product_id, how='inner')
        .select(
            PriceDF.product_id,
            sf.col("city_id"),
            sf.col("price")
    )
)

In [12]:
PriceJoinedDF.count()

131776

In [13]:
PriceJoinedDF.printSchema()
PriceJoinedDF.show()

root
 |-- product_id: integer (nullable = true)
 |-- city_id: integer (nullable = true)
 |-- price: float (nullable = true)

+----------+-------+-------+
|product_id|city_id|  price|
+----------+-------+-------+
|        34|    126| 684.46|
|       416|    274|   NULL|
|       338|     41|  12.81|
|       399|    103|  250.0|
|       319|    107| 513.21|
|       478|    162|   NULL|
|       402|    133|  155.0|
|        41|    223|  81.07|
|       208|    127|  495.6|
|       179|    277| 752.81|
|       216|    213| 164.65|
|       270|     93|3982.27|
|        66|     11|   NULL|
|       376|    174| 219.88|
|       486|    114|   NULL|
|        82|    237| 106.43|
|       287|    224| 374.16|
|       502|    227| 389.81|
|       241|    122|   NULL|
|       233|    145|  64.04|
+----------+-------+-------+
only showing top 20 rows



In [14]:
PriceStatDF = (
    PriceJoinedDF
        .groupBy("product_id")
        .agg(sf.max("price").alias("max_price"),
            sf.round(sf.avg("price"), 2).alias("avg_price"),
            sf.min("price").alias("min_price")
            )
)

In [15]:
PriceStatDF.printSchema()
PriceStatDF.show()

root
 |-- product_id: integer (nullable = true)
 |-- max_price: float (nullable = true)
 |-- avg_price: double (nullable = true)
 |-- min_price: float (nullable = true)



+----------+---------+---------+---------+
|product_id|max_price|avg_price|min_price|
+----------+---------+---------+---------+
|       496|    881.0|   281.19|     50.0|
|       463|    589.0|    306.4|    111.0|
|       471| 65901.11| 27692.07|   6000.0|
|       243| 11936.51|  3557.27|  1411.76|
|       392|  3847.08|  1000.81|   432.81|
|        31|   659.32|   386.71|    275.8|
|       451|  3911.52|   634.37|     61.0|
|       137|  14937.7|  8117.33|  3328.29|
|       251|  7155.84|  3403.97|   890.58|
|       458|   183.77|    105.2|    57.99|
|        65|  1319.07|   851.27|   597.38|
|        53|   406.35|   196.31|    97.62|
|       255|  3603.95|  1424.22|    643.6|
|       481|  92600.0| 27064.82|  8468.47|
|       296| 26666.99| 11591.51|   4709.8|
|       133|    245.0|    88.32|    39.93|
|       472|   935.25|   275.54|    106.0|
|       322| 750167.4|556312.04|468359.28|
|        78|  4932.42|  2143.22|   787.11|
|       513|     50.0|     6.44|     2.45|
+----------

In [16]:
# Сохранение результата на hdfs
(PriceStatDF
 .repartition(1)
 .write
 .mode("overwrite")
 .option("header", "true")
 .option("sep", ";")
 .csv(output_path1)
)

### Задание 2

In [17]:
UserDF = (
    spark.read
        .option("header", "false")
        .option("sep", "\t")
        .csv(demography_path)
)

CityRsDF = (
    spark.read
        .option("header", "false")
        .option("sep", "\t")
        .csv(city_rs_path)
)   

In [18]:
UserDF.printSchema()
UserDF.show()
CityRsDF.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)

+--------+-------------+-----+---+-----------+-------+----+
|     _c0|          _c1|  _c2|_c3|        _c4|    _c5| _c6|
+--------+-------------+-----+---+-----------+-------+----+
|16460783|1182885174073|  486|  2|10414533690|1078547|  85|
|16467391|1176953226317| 4669|  2|10414533690|1384327|  85|
|16467889|1169816093060| 6861|  2|10414533690|  33438|NULL|
|16468013|1172074147460| 5360|  2|10415971874|4000749|   1|
|16471027|1182034019343| 6311|  2|10414533690|3047069|  11|
|16474162|1173115608560| 5567|  2|10414533690|3868434|  36|
|16476386|1151057658677| 4351|  2|10414533690|3385314|  11|
|16481067|1177305838543| 3831|  2|10414533690|1112209|  13|
|16485965|1178801047573|-2369|  2|10415971874|2225871|  96|
|16486027|1163679273840| 5864

In [19]:
# Приводим типы и задаем названия полей.
UserDF = (
    UserDF
    .select(
        sf.col("_c0").cast(IntegerType()).alias("id"),
        sf.col("_c1").cast(LongType()).alias("create_date"),
        sf.col("_c2").cast(LongType()).alias("birth_date"),
        sf.col("_c3").cast(ByteType()).alias("gender"),
        sf.col("_c4").cast(IntegerType()).alias("id_country"),
        sf.col("_c5").cast(IntegerType()).alias("id_location"),
        sf.col("_c6").cast(IntegerType()).alias("id_region"),
    )
)
CityRsDF = (
    CityRsDF
    .select(
        sf.col("_c0").cast(IntegerType()).alias("ok_city_id"),
        sf.col("_c1").cast(IntegerType()).alias("rs_city_id")
    )
)

In [20]:
UserDF.printSchema()
UserDF.show()
CityRsDF.show()

root
 |-- id: integer (nullable = true)
 |-- create_date: long (nullable = true)
 |-- birth_date: long (nullable = true)
 |-- gender: byte (nullable = true)
 |-- id_country: integer (nullable = true)
 |-- id_location: integer (nullable = true)
 |-- id_region: integer (nullable = true)

+--------+-------------+----------+------+----------+-----------+---------+
|      id|  create_date|birth_date|gender|id_country|id_location|id_region|
+--------+-------------+----------+------+----------+-----------+---------+
|16460783|1182885174073|       486|     2|      NULL|    1078547|       85|
|16467391|1176953226317|      4669|     2|      NULL|    1384327|       85|
|16467889|1169816093060|      6861|     2|      NULL|      33438|     NULL|
|16468013|1172074147460|      5360|     2|      NULL|    4000749|        1|
|16471027|1182034019343|      6311|     2|      NULL|    3047069|       11|
|16474162|1173115608560|      5567|     2|      NULL|    3868434|       36|
|16476386|1151057658677|     

In [21]:
PriceJoinedDF.show()

+----------+-------+-------+
|product_id|city_id|  price|
+----------+-------+-------+
|        34|    126| 684.46|
|       416|    274|   NULL|
|       338|     41|  12.81|
|       399|    103|  250.0|
|       319|    107| 513.21|
|       478|    162|   NULL|
|       402|    133|  155.0|
|        41|    223|  81.07|
|       208|    127|  495.6|
|       179|    277| 752.81|
|       216|    213| 164.65|
|       270|     93|3982.27|
|        66|     11|   NULL|
|       376|    174| 219.88|
|       486|    114|   NULL|
|        82|    237| 106.43|
|       287|    224| 374.16|
|       502|    227| 389.81|
|       241|    122|   NULL|
|       233|    145|  64.04|
+----------+-------+-------+
only showing top 20 rows



In [22]:
PriceStatDF.show()

+----------+---------+---------+---------+
|product_id|max_price|avg_price|min_price|
+----------+---------+---------+---------+
|       496|    881.0|   281.19|     50.0|
|       463|    589.0|    306.4|    111.0|
|       471| 65901.11| 27692.07|   6000.0|
|       243| 11936.51|  3557.27|  1411.76|
|       392|  3847.08|  1000.81|   432.81|
|        31|   659.32|   386.71|    275.8|
|       451|  3911.52|   634.37|     61.0|
|       137|  14937.7|  8117.33|  3328.29|
|       251|  7155.84|  3403.97|   890.58|
|       458|   183.77|    105.2|    57.99|
|        65|  1319.07|   851.27|   597.38|
|        53|   406.35|   196.31|    97.62|
|       255|  3603.95|  1424.22|    643.6|
|       481|  92600.0| 27064.82|  8468.47|
|       296| 26666.99| 11591.51|   4709.8|
|       133|    245.0|    88.32|    39.93|
|       472|   935.25|   275.54|    106.0|
|       322| 750167.4|556312.04|468359.28|
|        78|  4932.42|  2143.22|   787.11|
|       513|     50.0|     6.44|     2.45|
+----------

In [24]:
# хотим видеть рядом с ценой еще и среднюю цену для сравнения, для этого такой join
PricePriceStatDF = (
    PriceDF.alias("left")
    .join(PriceStatDF, PriceDF.product_id == PriceStatDF.product_id, how='inner')
        .select(
            sf.col("left.product_id"),
            sf.col("city_id"),
            sf.col("price"),
            sf.col("avg_price")
        )
)

In [25]:
PricePriceStatDF.printSchema()
PricePriceStatDF.show()
PricePriceStatDF.count()

root
 |-- product_id: integer (nullable = true)
 |-- city_id: integer (nullable = true)
 |-- price: float (nullable = true)
 |-- avg_price: double (nullable = true)



+----------+-------+------+---------+
|product_id|city_id| price|avg_price|
+----------+-------+------+---------+
|       496|     89| 262.2|   281.19|
|       496|    180|169.48|   281.19|
|       496|    115|307.79|   281.19|
|       496|    124|182.53|   281.19|
|       496|     31|  NULL|   281.19|
|       496|     39|  NULL|   281.19|
|       496|    282| 559.0|   281.19|
|       496|    222|306.19|   281.19|
|       496|     41|  NULL|   281.19|
|       496|     62|285.74|   281.19|
|       496|    156|381.48|   281.19|
|       496|    224|386.55|   281.19|
|       496|    201| 376.7|   281.19|
|       496|    200|565.34|   281.19|
|       496|    145|315.39|   281.19|
|       496|    217|  NULL|   281.19|
|       496|      4|212.08|   281.19|
|       496|     56|  NULL|   281.19|
|       496|     26|269.59|   281.19|
|       496|    203|411.81|   281.19|
+----------+-------+------+---------+
only showing top 20 rows



131776

In [26]:
# выбираем теперь подходящие нам для статисктики города - в которых цена хоть на один продукт выше средней
CitiesForStatDF = (
    PricePriceStatDF.select(sf.col("city_id").alias("rs_city_id")).where(sf.col("price") > sf.col("avg_price")).distinct()
)

In [27]:
# добавляем названия городов
CitiesForStatDF = (
     CitiesForStatDF.alias("left")
    .join(CityDF, CitiesForStatDF.rs_city_id == CityDF.city_id, how='inner')
        .select(
            sf.col("left.rs_city_id"),
            sf.col("city"),
        )
)

In [28]:
CitiesForStatDF.show()

+----------+----------------+
|rs_city_id|            city|
+----------+----------------+
|       148|     Стерлитамак|
|       243|            Тара|
|        31|         Люберцы|
|       251|          Якутск|
|        85|         Гатчина|
|       137|      Карачаевск|
|        65|    Петрозаводск|
|       255|         Вилюйск|
|        53|          Тамбов|
|       133|        Малгобек|
|        78|       Череповец|
|       108|            Ялта|
|       155|Набережные Челны|
|        34|   Орехово-Зуево|
|       193|    Екатеринбург|
|       211|    Магнитогорск|
|       115|       Астрахань|
|       126|   г.Севастополь|
|       101|          Майкоп|
|        81|      Черняховск|
+----------+----------------+
only showing top 20 rows



In [29]:
# джоиним с rs_city для получения соотвествия id городов между OK и ROSSTAT
CitiesForStatDF = (
     CitiesForStatDF.alias("left")
    .join(CityRsDF, CitiesForStatDF.rs_city_id == CityRsDF.rs_city_id, how='inner')
        .select(
            sf.col("left.rs_city_id"),
            sf.col("city"),
            sf.col("ok_city_id")
        )
)
CitiesForStatDF.show()

+----------+----------------+----------+
|rs_city_id|            city|ok_city_id|
+----------+----------------+----------+
|       148|     Стерлитамак|    867411|
|       243|            Тара|   1078547|
|        31|         Люберцы|   1531654|
|       251|          Якутск|   2368818|
|        85|         Гатчина|    241372|
|       137|      Карачаевск|  12141837|
|        65|    Петрозаводск|    684283|
|       255|         Вилюйск|   2568145|
|        53|          Тамбов|   4332951|
|       133|        Малгобек|  12140731|
|        78|       Череповец|  12238521|
|       108|            Ялта|  12243061|
|       155|Набережные Челны|   3974068|
|        34|   Орехово-Зуево|    364123|
|       193|    Екатеринбург|   5176952|
|       211|    Магнитогорск|   3996003|
|       115|       Астрахань|  12146263|
|       126|   г.Севастополь|  12141538|
|       101|          Майкоп|   1024257|
|        81|      Черняховск|   1363646|
+----------+----------------+----------+
only showing top

In [30]:
# теперь наконец можем делать join с таблицей user по полю id_location
UserWithCityDF = (
    UserDF.alias("left")
    .join(CitiesForStatDF, UserDF.id_location == CitiesForStatDF.ok_city_id, how='inner')
        .select(
            sf.col("city"),
            sf.col("gender"),
            sf.col("birth_date"),
    )
)
UserWithCityDF.show()

+-----------+------+----------+
|       city|gender|birth_date|
+-----------+------+----------+
|Стерлитамак|     2|      7386|
|Стерлитамак|     1|     14099|
|Стерлитамак|     2|      1873|
|Стерлитамак|     2|      7017|
|Стерлитамак|     2|       681|
|Стерлитамак|     2|     -3617|
|Стерлитамак|     2|      5584|
|Стерлитамак|     2|      8792|
|Стерлитамак|     2|       512|
|Стерлитамак|     2|      4989|
|Стерлитамак|     1|      2340|
|Стерлитамак|     2|     11520|
|Стерлитамак|     2|      7907|
|Стерлитамак|     2|      7440|
|Стерлитамак|     2|      8438|
|Стерлитамак|     2|     10541|
|Стерлитамак|     2|     10159|
|Стерлитамак|     2|      9134|
|Стерлитамак|     2|      5833|
|Стерлитамак|     1|      7052|
+-----------+------+----------+
only showing top 20 rows



In [31]:
# теперь наконец считаем всю нужную статисктику
ok_dem = (
    UserWithCityDF.groupBy(sf.col("city"))
    .agg(
        sf.count(sf.col("birth_date")).alias("user_cnt"),
        sf.avg(sf.datediff(sf.lit(current_dt), sf.from_unixtime(sf.col("birth_date") * 24 * 3600)) / 365.25).cast(IntegerType()).alias("age_avg"),
        sf.count(sf.when(sf.col("gender") == 1, True)).alias("men_cnt"),
        sf.count(sf.when(sf.col("gender") == 2, True)).alias("women_cnt"),
    )
    .select(
        sf.col("city").alias("city_name"),
        sf.col("user_cnt"),
        sf.col("age_avg"),
        sf.col("men_cnt"),
        sf.col("women_cnt"),
        sf.round((sf.col("men_cnt") / sf.col("user_cnt")), 2).alias("men_share"),
        sf.round((sf.col("women_cnt") / sf.col("user_cnt")), 2).alias("women_share")
    )
    .orderBy(sf.col("user_cnt"), ascending=False)
)
ok_dem.show()

+-------------+--------+-------+-------+---------+---------+-----------+
|    city_name|user_cnt|age_avg|men_cnt|women_cnt|men_share|women_share|
+-------------+--------+-------+-------+---------+---------+-----------+
|         Тара|   34829|     40|   8176|    26653|     0.23|       0.77|
|      Городец|    4521|     40|   1412|     3112|     0.31|       0.69|
|         Юрга|    2124|     46|    596|     1528|     0.28|       0.72|
|       Волхов|    1963|     46|    477|     1486|     0.24|       0.76|
|         Чита|    1755|     43|    220|     1535|     0.13|       0.87|
|  Лесосибирск|    1245|     43|    255|      990|      0.2|        0.8|
|Краснокаменск|     903|     42|    290|      613|     0.32|       0.68|
|       Бердск|     845|     40|    279|      566|     0.33|       0.67|
|       Бежецк|     740|     42|     69|      671|     0.09|       0.91|
|        Певек|     676|     40|    194|      482|     0.29|       0.71|
|       Рязань|     589|     42|    176|      413| 

In [32]:
# Сохранение результата на hdfs
(ok_dem
 .repartition(1)
 .write
 .mode("overwrite")
 .option("header", "true")
 .option("sep", ";")
 .csv(output_path2)
)

### Задание 3

In [33]:
# найдем нужые значения
vals_for_stat = ok_dem.agg(
        sf.max(sf.col("age_avg")),
        sf.min(sf.col("age_avg")),
        sf.max(sf.col("men_share")),
        sf.max(sf.col("women_share"))).collect()[0]


In [34]:
# теперь выберем города под эти значения
CitiesForStatDF = (
    ok_dem.where( (sf.col("age_avg") == vals_for_stat[0]) | 
                    (sf.col("age_avg") == vals_for_stat[1]) |
                    (sf.col("men_share") == vals_for_stat[2]) |
                    (sf.col("women_share") == vals_for_stat[3]))
    .select(sf.col("city_name"))
)
CitiesForStatDF.show()

+------------+
|   city_name|
+------------+
|Новороссийск|
| Симферополь|
|      Тихвин|
|      Казань|
+------------+



In [35]:
# добавим id городов
CitiesForStatDF = (
    CitiesForStatDF.alias("left")
    .join(CityDF, CitiesForStatDF.city_name == CityDF.city, how='inner')
        .select(
            sf.col("city_id"),
            sf.col("city"),
    )
)
CitiesForStatDF.show()

+-------+------------+
|city_id|        city|
+-------+------------+
|    105| Симферополь|
|    112|Новороссийск|
|    153|      Казань|
|     88|      Тихвин|
+-------+------------+



In [36]:
# добавим продукты и цены на них
CitiesForStatPricesDF = (
    CitiesForStatDF.alias("left")
    .join(PriceJoinedDF, CitiesForStatDF.city_id == PriceDF.city_id, how='inner')
        .select(
            sf.col("left.city_id"),
            sf.col("city"),
            sf.col("product_id"),
            sf.col("price")
    )
)
CitiesForStatPricesDF.show()

+-------+-----------+----------+-------+
|city_id|       city|product_id|  price|
+-------+-----------+----------+-------+
|    105|Симферополь|       428|   NULL|
|    105|Симферополь|       512|  350.0|
|    105|Симферополь|       392|1069.12|
|    105|Симферополь|       454| 120.18|
|    105|Симферополь|       384|  13.06|
|    105|Симферополь|       348|  30.87|
|    105|Симферополь|       290|6117.42|
|    105|Симферополь|        36|1032.97|
|    105|Симферополь|        27|2201.38|
|    105|Симферополь|       126| 431.85|
|    105|Симферополь|        73| 873.73|
|    105|Симферополь|         2| 185.74|
|    105|Симферополь|        49| 501.73|
|    105|Симферополь|       361|  10.43|
|    105|Симферополь|       260| 712.37|
|    105|Симферополь|       442|  13.67|
|    105|Симферополь|       455|  84.33|
|    105|Симферополь|       430|   4.17|
|    105|Симферополь|       188| 169.65|
|    105|Симферополь|       478| 379.53|
+-------+-----------+----------+-------+
only showing top

In [37]:
# добавим названия продуктов
CitiesForStatPricesDF = (
    CitiesForStatPricesDF.alias("left")
    .join(ProductDF, CitiesForStatPricesDF.product_id == ProductDF.product_id, how='inner')
        .select(
            sf.col("price"),
            sf.col("city").alias("city_name"),
            sf.col("product")
    )
)
CitiesForStatPricesDF.show()

+-------+-----------+--------------------+
|  price|  city_name|             product|
+-------+-----------+--------------------+
|   NULL|Симферополь|Предоставление ме...|
|  350.0|Симферополь|Повторное получен...|
|1069.12|Симферополь|Регулировка разва...|
| 120.18|Симферополь|Водоснабжение гор...|
|  13.06|Симферополь|Земля для растени...|
|  30.87|Симферополь|Амброксол, 30 мг,...|
|6117.42|Симферополь|Велосипед для дош...|
|1032.97|Симферополь|  Масло оливковое, л|
|2201.38|Симферополь|Соленые и копчены...|
| 431.85|Симферополь|Обед в столовой, ...|
| 873.73|Симферополь|Чай черный байхов...|
| 185.74|Симферополь|Пельмени, манты, ...|
| 501.73|Симферополь|  Сыры плавленые, кг|
|  10.43|Симферополь|Сеннозиды А и B, ...|
| 712.37|Симферополь| Часы будильник, шт.|
|  13.67|Симферополь|Содержание, ремон...|
|  84.33|Симферополь|Газ сетевой, меся...|
|   4.17|Симферополь|Предоставление ме...|
| 169.65|Симферополь|  Трусы детские, шт.|
| 379.53|Симферополь|       Театры, билет|
+-------+--

In [38]:
from pyspark.sql.window import Window
# найдем максимальную и минимальную цену в каждом городе
product_stat = (
    CitiesForStatPricesDF
    .withColumn("cheap", sf.min("price").over(Window.partitionBy("city_name")))
    .withColumn("expensive", sf.max("price").over(Window.partitionBy("city_name")))
)
product_stat.show()        

+--------+---------+--------------------+-----+---------+
|   price|city_name|             product|cheap|expensive|
+--------+---------+--------------------+-----+---------+
|    NULL|   Казань|Газ сжиженный, ме...|  0.7| 909922.5|
|  167.64|   Казань|Панталоны, трусы ...|  0.7| 909922.5|
|   28.72|   Казань|Мыло хозяйственно...|  0.7| 909922.5|
|    NULL|   Казань|Бензин автомобильный|  0.7| 909922.5|
|    NULL|   Казань|Экскурсионная пое...|  0.7| 909922.5|
|  578.04|   Казань|           Какао, кг|  0.7| 909922.5|
|13649.88|   Казань|Шкаф для платья и...|  0.7| 909922.5|
|  386.05|   Казань|Ткань платьевая и...|  0.7| 909922.5|
| 3217.43|   Казань|Куртка женская бе...|  0.7| 909922.5|
|  860.93|   Казань|Выполнение работ ...|  0.7| 909922.5|
|  366.07|   Казань|     Маникюр, услуга|  0.7| 909922.5|
| 3007.19|   Казань|Шины для легковог...|  0.7| 909922.5|
|14577.38|   Казань|Обучение в негосу...|  0.7| 909922.5|
|  343.93|   Казань|Вино виноградное ...|  0.7| 909922.5|
|   51.33|   К

In [39]:
# считаем финальную статистику
product_stat = (
    product_stat
    .select(
        sf.col("city_name"),
        sf.col("price"),
        sf.when(sf.col("price") == sf.col("cheap"), sf.col("product")).alias("cheapest_product_name"),
        sf.when(sf.col("price") == sf.col("expensive"), sf.col("product")).alias("most_expensive_product_name"),
        (sf.col("expensive") - sf.col("cheap")).alias("price_difference")
    )
    .where( (sf.col("cheapest_product_name").isNotNull()) | (sf.col("most_expensive_product_name").isNotNull()))
    .groupBy(
        sf.col("city_name")
    )
    .agg (
        sf.last(sf.col("most_expensive_product_name"), True).alias("most_expensive_product_name"),
        sf.last(sf.col("cheapest_product_name"), True).alias("cheapest_product_name"),
        sf.last("price_difference", True).alias('price_difference')
    )
)
product_stat.show()

+------------+---------------------------+---------------------+----------------+
|   city_name|most_expensive_product_name|cheapest_product_name|price_difference|
+------------+---------------------------+---------------------+----------------+
|      Казань|       Легковой автомоби...| Предоставление ме...|        909921.8|
|Новороссийск|       Холодильник двухк...| Предоставление ме...|        32119.72|
| Симферополь|       Легковой автомоби...|      Спички, коробок|        790489.3|
|      Тихвин|       Легковой автомоби...| Предоставление ме...|       619285.94|
+------------+---------------------------+---------------------+----------------+



In [40]:
# сохраняем результаты
(product_stat
 .repartition(1)
 .write
 .mode("overwrite")
 .option("header", "true")
 .option("sep", ";")
 .csv(output_path3)
)

In [50]:
price_stat_path = "/user/a.marchenko/task4/price_stat/"
PriceStatDF = spark.read.option("header", "true").csv(price_stat_path, sep=';')
PriceStatDF = (
    PriceStatDF
    .select(
        sf.col("product_id").cast(IntegerType()),
        sf.col("max_price").cast(FloatType()),
        sf.col("avg_price").cast(FloatType()),
        sf.col("min_price").cast(FloatType())
    )
)
PriceStatDF.printSchema()
PriceStatDF.show()

root
 |-- product_id: integer (nullable = true)
 |-- max_price: float (nullable = true)
 |-- avg_price: float (nullable = true)
 |-- min_price: float (nullable = true)

+----------+---------+---------+---------+
|product_id|max_price|avg_price|min_price|
+----------+---------+---------+---------+
|       471| 65901.11| 27692.07|   6000.0|
|       496|    881.0|   281.19|     50.0|
|       463|    589.0|    306.4|    111.0|
|       392|  3847.08|  1000.81|   432.81|
|       243| 11936.51|  3557.27|  1411.76|
|        31|   659.32|   386.71|    275.8|
|       451|  3911.52|   634.37|     61.0|
|       137|  14937.7|  8117.33|  3328.29|
|       251|  7155.84|  3403.97|   890.58|
|        65|  1319.07|   851.27|   597.38|
|       458|   183.77|    105.2|    57.99|
|       255|  3603.95|  1424.22|    643.6|
|        53|   406.35|   196.31|    97.62|
|       481|  92600.0| 27064.82|  8468.47|
|       472|   935.25|   275.54|    106.0|
|       133|    245.0|    88.32|    39.93|
|       296| 2

In [51]:
# После работы обязательно отключаем спарк и отдаем ресурсы!
spark.stop()